In [1]:
from datetime import datetime
import pandas
import ccxt

def price_fetcher(event,context):
    '''
        Stateless Function to fetch latest price data of all stocks
        Event/Trigger: Cloud Scheduler

    '''


    '''
        Stock List to be fetched from GET /stocks
    '''
    stock_list=['BTCUSDT','ETHUSDT','BNBUSDT']


    '''Hard Coded Parameters: 
            Number of Candles, 
            Interval, 
            User ID/Pwd, 
            Whether to Fetch the Last Price 
    '''
    candles = 50
    interval = '1h'
    exchange= ccxt.binance({
                        'apiKey': 'INH9JYsd4Cu3kMPoONiCVHP3KlACsg3F4ehDN1cburoKohsARMpZGcq4PnQoqzyF',
                        'secret': 'FSVMXANswsGOj3B4Oi4NSDOlX5fsvWOJ3s56DQsWvJTjLhSuPyq1aFLbFEWoOrMt',
                        'enableRateLimit': True, 
                        'options': {'defaultType': 'future'},
                        'hedgeMode':True
                        })
    last_incomplete_candle = False


    '''
        For Each Stock: Store the fetched Price Values and Datetimes as a Pandas Dataframe 
    '''
    df_dict = dict()
    for symbol in stock_list:    
        if last_incomplete_candle == False:
            closes = [[datetime.utcfromtimestamp(float(elem[0]) / 1000.),elem[4]] 
            for elem in exchange.fapiPublic_get_klines({'symbol':symbol,'interval':interval})][-candles:-1]
        if last_incomplete_candle == True:
            closes = [[datetime.utcfromtimestamp(float(elem[0]) / 1000.),elem[4]] 
            for elem in exchange.fapiPublic_get_klines({'symbol':symbol,'interval':interval})][-(candles-1):]
        dates = [elem[0] for elem in closes]
        values = [float(elem[1]) for elem in closes]
        df = pandas.DataFrame([(elem[0],elem[1]) for elem in zip(dates,values)],columns=['datetime','closes'])
        df_dict[symbol]=df
    

    '''
    update through PUT /stocks/<stock_id>
    '''
    return df_dict
    

In [2]:
store_dict = price_fetcher(None,None)

In [6]:
store_dict

{'BTCUSDT':               datetime   closes
 0  2022-03-21 12:00:00  41203.1
 1  2022-03-21 13:00:00  41085.0
 2  2022-03-21 14:00:00  41016.7
 3  2022-03-21 15:00:00  41198.3
 4  2022-03-21 16:00:00  40768.6
 5  2022-03-21 17:00:00  41035.7
 6  2022-03-21 18:00:00  40924.3
 7  2022-03-21 19:00:00  41168.0
 8  2022-03-21 20:00:00  41104.3
 9  2022-03-21 21:00:00  41278.0
 10 2022-03-21 22:00:00  41098.1
 11 2022-03-21 23:00:00  40982.0
 12 2022-03-22 00:00:00  41106.5
 13 2022-03-22 01:00:00  41148.9
 14 2022-03-22 02:00:00  41815.9
 15 2022-03-22 03:00:00  42230.0
 16 2022-03-22 04:00:00  42992.4
 17 2022-03-22 05:00:00  42850.0
 18 2022-03-22 06:00:00  42241.5
 19 2022-03-22 07:00:00  42197.9
 20 2022-03-22 08:00:00  42535.8
 21 2022-03-22 09:00:00  42448.0
 22 2022-03-22 10:00:00  42802.7
 23 2022-03-22 11:00:00  42977.0
 24 2022-03-22 12:00:00  42937.9
 25 2022-03-22 13:00:00  42816.2
 26 2022-03-22 14:00:00  42739.5
 27 2022-03-22 15:00:00  42567.2
 28 2022-03-22 16:00:00  42506.9